In [1]:
import os
import psycopg2
from supabase import create_client, Client
from dotenv import load_dotenv
from pathlib import Path
import datetime


# Load environment variables from .env file
env_path = Path('__file__').parent.resolve().parent / '.env'
if env_path.exists():
    load_dotenv(dotenv_path=env_path)
else:
    print('No .env file found.')

In [2]:
def serialize_datetimes(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    elif isinstance(obj, datetime.date):
        return obj.isoformat()  # This handles date objects
    elif isinstance(obj, dict):
        return {k: serialize_datetimes(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [serialize_datetimes(item) for item in obj]
    else:
        return obj



In [3]:
def transfer_table(table_name: str):
    # Fetch data from local DB
    with local_conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {table_name}")
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    if not rows:
        print(f"No data found in table {table_name}")
        return

    # Convert each row into a dictionary {column: value}
    records = [serialize_datetimes(dict(zip(columns, row))) for row in rows]

    # Use the Supabase Python connector to insert data into the table
    response = supabase_conn.table(table_name).insert(records).execute()

    # Instead of checking response.error, use raise_when_api_error() to check for errors.
    try:
        response.raise_when_api_error(response.data)
    except Exception as e:
        print(f"Error inserting into {table_name}: {e}")
    else:
        print(f"Successfully transferred {len(records)} rows into {table_name}")


In [4]:
local_conn = psycopg2.connect(
    host="localhost",
    dbname="Kipu",
    user="postgres",
    password="#Lemontree501",
    port="5432"
)

In [5]:
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
supabase_conn = create_client(url, key)

In [6]:
# List of tables to transfer
tables = ['admin_discharge', 'ama_forms', 'detox_forms', 'calendar', 'latest', 'program_history', 'ref_discharge_types', 'ref_program_types'] 

for table in tables:
    transfer_table(table)

# Close connections
local_conn.close()

Successfully transferred 286 rows into admin_discharge
Successfully transferred 1090 rows into ama_forms
Successfully transferred 1604 rows into detox_forms
Successfully transferred 4018 rows into calendar
Successfully transferred 5708 rows into latest
Successfully transferred 15374 rows into program_history
Successfully transferred 34 rows into ref_discharge_types
Successfully transferred 29 rows into ref_program_types


AttributeError: 'SyncClient' object has no attribute 'close'